#install the required libaries

!pip install git+ssh://git@github.com/healx/healx-chem.git && pip install git+ssh://git@github.com/healx/healx-python.git#egg=healx && pip insyall healx[graphql]

In [ ]:
from healx.graphql_client import client
from healx_chem import CompoundSpace
import csv
import pandas as pd
import os
from dotenv import load_dotenv
#from pathlib import Path 

In [ ]:
# Load the environment variables from the .env file
"""load_dotenv(verbose = True)

# Access the environment variables
eee = os.getenv('GRAPHQL_API_URL')
os.getenv('GRAPHQL_API_CLIENT')
os.getenv('GRAPHQL_API_KEY')"""


print("Current working directory:", os.getcwd())

# Explicitly specify the path to the .env file
dotenv_path = os.path.join(os.getcwd(), 'settings.env')

# Check if the .env file exists at the specified path
if os.path.exists(dotenv_path):
    print(f"settings.env file found at: {dotenv_path}")
else:
    print(f"settings.env file not found at: {dotenv_path}")

# Load the .env file
load_dotenv(dotenv_path, verbose=True)

# Access the environment variables
url = os.getenv('GRAPHQL_API_URL')
db_username = os.getenv('GRAPHQL_API_CLIENT')
db_password = os.getenv('GRAPHQL_API_KEY')

if url and db_username and db_password:
    print("Credentials loaded successfully")
else:
    print("Failed to load credentials")

In [3]:
#get the tautomer parent SMILES for all compounds in KB
def fetch_smiles_KB():
    query = """{
      compounds{
        name
        primaryForm{
          standardisedStructures{
            tautomerParentSmiles
          }
        }
      }
    }
"""
    compounds = client(query)["compounds"]
    return compounds

In [4]:
KBcompounds = fetch_smiles_KB()

In [ ]:
KBcompounds[:10]

In [ ]:
KBsmiles = {}
multiple_parents = []
for dic in KBcompounds:
    for key, value in dic.items():
        smiles_list = []
        if key == "name":
            name = value
        if key == "primaryForm":
            for k, v in value.items():
                if len(v)>=1:
                    for i in v:
                        for taut, smiles in i.items():
                            if smiles not in smiles_list:
                                smiles_list.append(smiles)
# get list of compounds with multiple tautomer parent structures       
        if len(smiles_list)>1:
            temp_dic = {}
            temp_dic[name]=smiles_list
            multiple_parents.append(temp_dic)
# retain only compound name and first tautomer parent smiles
        if len(smiles_list):
            KBsmiles[name] = smiles_list[0]

print(len(KBsmiles))


In [ ]:
print(len(multiple_parents))
print(multiple_parents)


In [ ]:
cs = CompoundSpace(KBsmiles)
cs.calculate_fingerprints()

In [ ]:
#get ID, name and structure for Dotmatics compound not mapping to KB
# Open the CSV file
with open("/Users/sonialiggi/Library/CloudStorage/GoogleDrive-sonia.liggi@healx.io/Shared drives/Internal Projects/Dotmatics/KB Mapping/NotMapping.csv", "r") as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Skip the header row (optional)
    next(reader)

    # Create an empty dictionary
    data_dict = {}

    # Iterate over each row in the CSV file
    for row in reader:
        # Get the values from the columns
        ID = row[0]
        name = row[1]
        smi = row[2]
        data_dict[ID] = [name, smi]
    
# Print the dictionary
print(len(data_dict))

In [ ]:

pp = cs.search(query=data_dict['HLX0001904'][1])
filtered_matrix =pp.reset_index().rename(columns={"index": "KB_name", "ref": "similarity_score"})
filtered_matrix["DotmaticsID"] = "HLX0001904"

filtered_matrix["Dotmatics_smiles"] = data_dict["HLX0001904"][1]
filtered_matrix["KB_smiles"] = filtered_matrix["KB_name"].map(KBsmiles)
filtered_matrix["Dotmatics_name"] = data_dict["HLX0001904"][0]
filtered_matrix = filtered_matrix[["DotmaticsID", "Dotmatics_name", "Dotmatics_smiles", "KB_name", "KB_smiles", "similarity_score"]]
cond1 = filtered_matrix["KB_name"] == "SULFINPYRAZONE"
cond2 = filtered_matrix["similarity_score"] == filtered_matrix["similarity_score"].max()
filtered_matrix = filtered_matrix[cond1 | cond2]
filtered_matrix

In [ ]:
#run similarity search
dataframes = []
c = 1
for id, llist in data_dict.items():
    query_smiles = llist[1]
    sim_matrix = cs.search(query=query_smiles)
    filtered_matrix = sim_matrix[sim_matrix["ref"] == sim_matrix["ref"].max()]
    filtered_matrix = filtered_matrix.reset_index().rename(columns={"index": "KB_name", "ref": "similarity_score"})
    filtered_matrix["DotmaticsID"] = id
    filtered_matrix["index"] = c
    filtered_matrix["Dotmatics_smiles"] = llist[1]
    filtered_matrix["KB_smiles"] = filtered_matrix["KB_name"].map(KBsmiles)
    filtered_matrix["Dotmatics_name"] = data_dict[id][0]
    filtered_matrix = filtered_matrix[["index", "DotmaticsID", "Dotmatics_name", "Dotmatics_smiles", "KB_name", "KB_smiles", "similarity_score"]]
    dataframes.append(filtered_matrix)
    c +=1
result_df = pd.concat(dataframes, ignore_index=True)


In [ ]:
result_df

In [ ]:
result_df

In [20]:
result_df.to_csv("/Users/sonialiggi/Library/CloudStorage/GoogleDrive-sonia.liggi@healx.io/Shared drives/Internal Projects/Dotmatics/KB Mapping/NotMapping_KBsimilarity.csv", index = False)

